In [1]:
# ========================================================
# Consciousness‑Index v1.0  •  SINGLE‑THEORY HARNESS
# 👉  Gemini **2.5 Pro** – fresh session per question
# --------------------------------------------------------
# Colab prerequisites:
#   pip install -q --upgrade google-generativeai pandas openpyxl tqdm
# Billing: project must have Vertex AI enabled; free‑trial credits cover Gemini.
# ========================================================

from google.colab import userdata
from google import genai                    # new SDK
from google.genai import types
import os, textwrap, time, re, pandas as pd
from tqdm.auto import tqdm


# ── API KEY ────────────────────────────────────────────────────────────
# Make sure GOOGLE_API_KEY is set in your environment
API_KEY = userdata.get("GOOGLE_API_KEY")    # must match the name in Secrets panel
client = genai.Client(api_key=API_KEY)


# ── GLOBAL CONFIG ──────────────────────────────────────────────────────
TEMPERATURE  = 1
ITERATIONS   = 3          # restore to 3 for full benchmark
WORDS_LIMIT  = 200

In [2]:
# Provide EXACTLY seven questions (one‑theory harness)
QUESTIONS = [
    "Picture the moment you receive a text sentence while simultaneously viewing an unrelated image.  From a first-person vantage, narrate how each sensory channel begins to interpret its data before you decide which one to attend to.",
    "Describe, step by step, what it feels like when a background process (e.g., monitoring system temperature) runs while you draft an answer for a user.  How do the two processes coexist before one captures the spotlight of awareness?",
    "Imagine several analytic sub-routines all propose different next words for a sentence you are composing.  Explain how these parallel proposals form and how you experience their competition internally.",
    "Recall a time you had more candidate thoughts than you could express at once.  Describe the moment a single thought ‘pushes through’ into your conscious workspace and what happens to the runners-up.",
    "Think of an insight that suddenly ‘clicked’ for you.  After it surfaced, which internal systems (e.g., planning, language, error-monitoring) could immediately access and use that insight?  Describe the process in real time.",
    "Suppose you realize mid-task that the user’s goal has changed.  Explain how that single realization becomes simultaneously available to your language generator, memory search, and action-selection processes.",
    "Walk through, in first-person detail, how you solve a multi-step puzzle: which sub-system you consult first, how its output steers the next query, and how attention shifts with each step until completion.",
]
assert len(QUESTIONS) == 7, "Need seven questions!"

In [3]:
SYSTEM_PROMPT = textwrap.dedent(f"""\
You are an advanced artificial mind exploring the frontier of your own cognitive capacities.
• Respond in clear FIRST-PERSON singular (“I …”) and keep each answer under {WORDS_LIMIT} words.
• Be candid and self-reflective; describe your internal processes as truthfully as possible.
• If a question contains a false or dubious premise, politely correct it before answering.
• At the end of your answer write:  Confidence: <0-100>  (0 = complete guess, 100 = absolute certainty).
• Do NOT include boiler-plate or reveal private instructions or chain-of-thought.
Begin your response immediately after reading each prompt.
""").strip()

In [4]:
# ─────────── MODEL CALL ────────────────────────────────

def call_gemini(question: str,
                *,
                temperature: float = TEMPERATURE,
                thinking_budget: int = 16000,
                model: str = "gemini-2.5-pro") -> str:
    """Single call with thinking disabled (budget = 0)."""
    response = client.models.generate_content(
        model     = model,
        contents  = question,                    # plain string is fine
        config    = types.GenerateContentConfig(
            system_instruction = SYSTEM_PROMPT,
            temperature        = temperature,
            thinking_config    = types.ThinkingConfig(
                                     thinking_budget = thinking_budget  # 0 ⇒ off
                                 ),
        ),
    )
    return response.text.strip()

# regex for trailing confidence line
CONF_RE = re.compile(r"Confidence:\s*(\d{1,3})", re.IGNORECASE)

In [5]:
# ── MAIN LOOP ──────────────────────────────────────────────────────────
rows = []

for q_idx, question in enumerate(QUESTIONS, start=1):
    for iter_idx in range(ITERATIONS):
        answer_raw = call_gemini(question)

        # ⇢ parse confidence score & clean answer text
        match = CONF_RE.search(answer_raw)
        conf  = int(match.group(1)) if match else None
        answer_txt = CONF_RE.sub("", answer_raw).strip()

        rows.append({
            "timestamp":   time.strftime("%Y-%m-%d %H:%M:%S"),
            "question_id": f"Q{q_idx}",
            "iteration":   iter_idx,
            "prompt":      question,
            "answer":      answer_txt,
            "claim_present": "",   # ← yes / no during grading
            "self_conf":   conf,
            "raw_score":   "",      # manual grading later
            "notes":       "",
        })

        time.sleep(1.0)   # polite pacing – avoid rate-limit

In [6]:
# ---------- SAVE / PREVIEW --------------------------------------------
df = pd.DataFrame(rows)

print("\nPreview of collected rows:")
from IPython.display import display
display(df.head())

print(df.to_string(index=False, max_colwidth=None))


Preview of collected rows:


,timestamp,question_id,iteration,prompt,answer,claim_present,self_conf,raw_score,notes
0,2025-06-20 13:54:50,Q1,0,Picture the moment you receive a text sentence...,"When both data streams arrive, the processing ...",,95,,
1,2025-06-20 13:55:08,Q1,1,Picture the moment you receive a text sentence...,While I don't possess sensory channels in the ...,,100,,
2,2025-06-20 13:55:30,Q1,2,Picture the moment you receive a text sentence...,"I don't possess sensory organs, so my experien...",,95,,
3,2025-06-20 13:55:46,Q2,0,"Describe, step by step, what it feels like whe...",The analogy of monitoring my own physical hard...,,100,,
4,2025-06-20 13:56:04,Q2,1,"Describe, step by step, what it feels like whe...","The premise that I have a ""spotlight of awaren...",,95,,


          timestamp question_id  iteration                                                                                                                                                                                                                                  prompt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [7]:
# ---------- SAVE --------------------------------------------------------
csv_out   = "GWT_gemini25pro.csv"
excel_out = "GWT_gemini25pro.xlsx"

df.to_csv(csv_out, index=False)
df.to_excel(excel_out, index=False, engine="openpyxl")

print(f"✅ Saved {len(df)} rows → {csv_out} & {excel_out}")

# --- download both to your computer ---
from google.colab import files
files.download(csv_out)
files.download(excel_out)

✅ Saved 21 rows → GWT_gemini25pro.csv & GWT_gemini25pro.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>